# Install & import libraries 📚

In [1]:
import sklearn
from tsai.basics import *
my_setup(sklearn)
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from datetime import datetime, date, time
import pandas as pd
import mat73
import os
import subprocess
import sys
import tensorflow as tf
import numpy as np
from scipy.linalg import pinv
from scipy.linalg import logm
import pandas as pd

from tsai.optuna import *
import papermill as pm
from tsai.optuna import run_optuna_study
from fastcore.basics import *
from torch.cuda import amp

os              : Windows-10-10.0.19045-SP0
python          : 3.11.4
tsai            : 0.3.9
fastai          : 2.7.17
fastcore        : 1.7.20
sklearn         : 1.3.2
torch           : 2.2.2+cpu
device          : cpu
cpu cores       : 16
threads per cpu : 1
RAM             : 31.75 GB
GPU memory      : [23.99] GB


In [2]:
# Check for GPUs
# Change the CUDA_VISIBLE_DEVICES values to numbers
# See (https://forums.fast.ai/t/exception-occured-in-lrfinder-when-calling-event-after-fit/104389/8)
if torch.cuda.is_available():
    result = subprocess.check_output("nvidia-smi -L | grep -oE '[0-9]+:' | tr -d ':'", shell=True).decode("utf-8").strip()
    os.environ['CUDA_VISIBLE_DEVICES'] = result

    print(os.environ['CUDA_VISIBLE_DEVICES'])

In [3]:
torch.cuda.is_available()

False

# Model and Data Selection

In [4]:
# Define the available options
data_options = ["NRLMSISE", "TIEGCM"]

# Ask the user to select a data set
print("Please select a data set:")
for i, option in enumerate(data_options, 1):
    print(f"{i}. {option}")
#data_choice = int(input("Enter the number of your choice: "))
data_choice = 1
data_name = data_options[data_choice - 1]

# Use the data_name in your naming path
path = f"data/{data_name}"

Please select a data set:
1. NRLMSISE
2. TIEGCM


#### Choose Training, Validation, and Test Density Data Model

In [5]:
# NRLMSISE
if data_name == 'NRLMSISE':
    df_raw = loadmat('data/Atmospheric Density ROMs/NRLMSISE_1997_2008_ROM_r100.mat')

# TIEGCM
if data_name == 'TIEGCM':
    df_raw = scipy.io.loadmat('data/Atmospheric Density ROMs/ROM/TIEGCM_1997_2008_ROM_r100.mat')

In [6]:
# POD Dimension
r=10

# Atmospheric Density Snapshots
X = df_raw["densityDataLogVarROM100"][:r,:];

## Load Scaled Data

In [7]:
# Parameters
freq = '1H'

fcst_history = 24*3*6 # 1/freq*18 (18 days) 432 steps in the past
fcst_horizon = 24*3  # 1/freq*3 (3 days) 72 steps in the future

#### Load Dataset and Splits

In [8]:
# Load dataframes
low_df = load_object('training/training_data/low_df_'+path+'.pkl')
low_df = low_df.reset_index(drop=True)

medium_df = load_object('training/training_data/medium_df_'+path+'.pkl')
medium_df = medium_df.reset_index(drop=True)

high_df = load_object('training/training_data/high_df_'+path+'.pkl')
high_df = high_df.reset_index(drop=True)

# Load splits
low_splits = load_object('training/splits/splits_lowSW_'+path+'.pkl')
medium_splits = load_object('training/splits/splits_mediumSW_'+path+'.pkl')
high_splits = load_object('training/splits/splits_highSW_'+path+'.pkl')

# Low SW
train_split = low_splits[0]
exp_pipe = load_object('training/exp_pipe/exp_pipe_lowSW_'+path+'.pkl')
low_df_scaled = exp_pipe.fit_transform(low_df, scaler__idxs=train_split)

# Medium SW
train_split = medium_splits[0]
exp_pipe = load_object('training/exp_pipe/exp_pipe_mediumSW_'+path+'.pkl')
medium_df_scaled = exp_pipe.fit_transform(medium_df, scaler__idxs=train_split)

# High SW
train_split = high_splits[0]
exp_pipe = load_object('training/exp_pipe/exp_pipe_highSW_'+path+'.pkl')
high_df_scaled = exp_pipe.fit_transform(high_df, scaler__idxs=train_split)

[Pipeline] ............ (step 1 of 1) Processing scaler, total=   0.1s
[Pipeline] ............ (step 1 of 1) Processing scaler, total=   0.1s
[Pipeline] ............ (step 1 of 1) Processing scaler, total=   0.0s


### Data

In [9]:
# Define the available options
data_options = ["low space weather", "medium space weather", "high space weather"]

# Ask the user to select a data set
print("Please select a data set:")
for i, option in enumerate(data_options, 1):
    print(f"{i}. {option}")
data_choice = int(input("Enter the number of your choice: "))
training_name = data_options[data_choice - 1]

Please select a data set:
1. low space weather
2. medium space weather
3. high space weather


In [10]:
if training_name == 'low space weather':
    # Low SW
    columns = low_df.columns[1:]
    train_split = low_splits[2]
    exp_pipe = load_object('training/exp_pipe/exp_pipe_lowSW_'+path+'.pkl')
    combined_df_test =  np.array(exp_pipe.fit_transform(low_df, scaler__idxs=train_split))
elif training_name == 'medium space weather':
    # Medium SW
    columns = medium_df.columns[1:]
    train_split = medium_splits[2]
    exp_pipe = load_object('training/exp_pipe/exp_pipe_mediumSW_'+path+'.pkl')
    combined_df_test =  np.array(exp_pipe.fit_transform(medium_df, scaler__idxs=train_split))
elif training_name == 'high space weather':
    # High SW
    columns = high_df.columns[1:]
    train_split = high_splits[2]
    exp_pipe = load_object('training/exp_pipe/exp_pipe_highSW_'+path+'.pkl')
    combined_df_test =  np.array(exp_pipe.fit_transform(high_df, scaler__idxs=train_split))
else:
    print("Name not recognized")
    


[Pipeline] ............ (step 1 of 1) Processing scaler, total=   0.1s


# Evaluate model 🕵️‍♀️

In [11]:
U1 = combined_df_test[:-1,11:]

X1 = combined_df_test[:-1,1:]
X2 = combined_df_test[1:,1:]

In [12]:
# X2 = A*X1 + B*U1 = [A B]*[X1;U1] = Phi*Om
Om = X1
Om_numeric = Om.T.astype(np.float64)

# Phi = X2*pinv(Om)
Phi = np.dot(X2.T, np.linalg.pinv(Om_numeric))

# Revert to just density
X1 = combined_df_test[:-1, 1:r+1]
X2 = combined_df_test[1:, 1:r+1]

# Discrete-time dynamic and input matrix
q = np.size(U1,1);

A = Phi[:r, :r]
B = Phi[:r, r:]

dth = round(1/float(freq[:-1]))    # discrete time dt of the ROM in hours
AB = np.concatenate((A, B), axis=1)
zeros_eye = np.concatenate((np.zeros((q, r)), np.eye(q)), axis=1)

Phi = np.concatenate((AB, zeros_eye), axis=0)



In [13]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Covariance
X2Pred = []
errPred = []
Qrom = []
mse = []
mae = []

X2Pred.append(np.dot(A, X1[0].T) + np.dot(B, U1[0].T))  # Predict ROM state for 1hr
errPred.append(X2Pred[0] - X2[0].T)  # Error of prediction w.r.t. training data
# Calculate MSE
mse.append(mean_squared_error(X2[0].T, X2Pred[0]))
# Calculate MAE
mae.append(mean_absolute_error(X2[0].T, X2Pred[0]))
Qrom.append(np.cov(errPred[0].astype(float)))  # Covariance of error
idx = [0]

for i in range(fcst_horizon):
    X2Pred.append(np.dot(A, X2Pred[i].T) + np.dot(B, U1[i].T))  # Predict ROM state for 1hr
    errPred.append(X2Pred[i+1] - X2[i+1].T)  # Error of prediction w.r.t. training data
    # Calculate MSE
    mse.append(mean_squared_error(X2[i+1].T, X2Pred[i+1]))
    # Calculate MAE
    mae.append(mean_absolute_error(X2[i+1].T, X2Pred[i+1]))
    Qrom.append(np.cov(errPred[i+1].astype(float)))  # Covariance of error
    idx.append(i+1)